In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import pairwise

In [2]:
import sys
sys.path.append('/Users/bengikoseoglu/anaconda2/pkgs')
sys.path.append('/Users/bengikoseoglu/anaconda2/pckgs')
sys.path.append('/Users/bengikoseoglu')
sys.path.append('/Library/Python/2.7/site-packages/')
sys.path.append('/Users/bengikoseoglu/anaconda2/lib/python2.7/site-packages/nltk')
sys.path.append('/Users/bengikoseoglu/nltk_data')
sys.path

['',
 '/Users/bengikoseoglu/anaconda2/lib/python27.zip',
 '/Users/bengikoseoglu/anaconda2/lib/python2.7',
 '/Users/bengikoseoglu/anaconda2/lib/python2.7/plat-darwin',
 '/Users/bengikoseoglu/anaconda2/lib/python2.7/plat-mac',
 '/Users/bengikoseoglu/anaconda2/lib/python2.7/plat-mac/lib-scriptpackages',
 '/Users/bengikoseoglu/anaconda2/lib/python2.7/lib-tk',
 '/Users/bengikoseoglu/anaconda2/lib/python2.7/lib-old',
 '/Users/bengikoseoglu/anaconda2/lib/python2.7/lib-dynload',
 '/Users/bengikoseoglu/anaconda2/lib/python2.7/site-packages',
 '/Users/bengikoseoglu/anaconda2/lib/python2.7/site-packages/aeosa',
 '/Users/bengikoseoglu/anaconda2/lib/python2.7/site-packages/IPython/extensions',
 '/Users/bengikoseoglu/.ipython',
 '/Users/bengikoseoglu/anaconda2/pkgs',
 '/Users/bengikoseoglu/anaconda2/pckgs',
 '/Users/bengikoseoglu',
 '/Library/Python/2.7/site-packages/',
 '/Users/bengikoseoglu/anaconda2/lib/python2.7/site-packages/nltk',
 '/Users/bengikoseoglu/nltk_data']

In [3]:
bow_product_catalog=pd.read_csv("cameras_catalog_2.csv",sep=';',dtype={'price': 'S10','optical_zoom':'S10','unit':'S10','width':'S10',
 'height':'S10',
 'depth':'S10',
 'unit2':'S10',
 'value':'S10',
'iso_range':'S10','video_resolution':'S10','megapixels':'S10','battery':'S10','has_wifi':'S10','has_bluetooth':'S10'})#actuals that are in the product catalog
cam_gold_standard=pd.read_csv("cameras_gold_standard_final.csv")#matchings

In [4]:
bow_product_catalog.head()

,id,id_2,name,brand,type,color,itemCondition,availability,priceCurrency,price,...,unit,width,height,depth,unit2,value,model,has_wifi,has_bluetooth,battery
0,walmart-51650ba6d92bdca372e07f63cf51df06,camera_001,Nikon D3400 Digital SLR Camera with 24.2 Megap...,Nikon,DSLR,Black,NewCondition,InStock,USD,569.99,...,Inches,6.5,5.32,11.02,lbs,4.03,D3400,Yes,yes,EN-EL14A r
1,walmart-698daed2260ea72556cb5cb342d17f78,camera_002,Canon EOS Rebel T6 Digital SLR Camera with 18-...,Canon,DSLR,Black,NewCondition,InStock,USD,449.99,...,inches,5.1,4,3.1,lbs,15.52,1159C003,yes,yes,1 x Canon
2,samys-ad91d35d56d16a8d759dc41c692a0b9c,camera_003,Sony Alpha a68 Digital SLR Camera with DT 18-5...,Sony,DSLR,Black,NewCondition,InStock,USD,698,...,in,5.61,4.1,3.26,g,696,a68,no,no,NP-FM500H
3,samys-63325293c96d4d030868ce8ca9fbc7b7,camera_004,Pentax K-1 Mark II Digital SLR Camera Body,Pentax,DSLR,Black,NewCondition,InStock,USD,"1,996.95",...,In,5.4,4.3,3.4,g,925,K-1 Mark II,yes,no,D-LI90 Rec
4,bestbuy-cba5da0c30ab48b7f2303cc9734ae8c7,camera_005,Panasonic - LUMIX DC-ZS70 20.3-Megapixel Digit...,Panasonic,DSLR,Black,NewCondition,InStock,USD,449.99,...,inches,4.41,2.65,1.62,pounds,0.71,DC-ZS70K,yes,no,1 x Li-ion


In [5]:
cam_gold_standard.head()

,page_id,catalog_id,label
0,argos-23426c3985d6e24911336f3d8106409b,camera_001,0
1,argos-23426c3985d6e24911336f3d8106409b,camera_002,0
2,argos-23426c3985d6e24911336f3d8106409b,camera_003,0
3,argos-23426c3985d6e24911336f3d8106409b,camera_004,0
4,argos-23426c3985d6e24911336f3d8106409b,camera_005,0


In [6]:
cam_gold_standard.shape

(6000, 3)

In [7]:
cam_gold_standard['label'].sum()

118L

- We need to get information of page ids from extracted information.

In [11]:
os.chdir('/Users/bengikoseoglu/Documents/Masters/Semester2/TeamProject/Product_Classification/Identity_resolution/options/cameras/cameras_products')

In [12]:
array_list=os.listdir(os.getcwd())
array_list

['robertscamera.csv',
 'samys.csv',
 'result_pre.csv',
 'wilkinson.csv',
 '.DS_Store',
 'uniquephoto.csv',
 'cliftoncameras.csv',
 'focuscamera.csv',
 'walmart.csv',
 'bhphotovideo.csv',
 'argos.csv',
 'pictureline.csv',
 'flipkart.csv',
 'cameta.csv',
 'currys.csv',
 'jessops.csv',
 'newegg.csv',
 'bestbuy.csv',
 'buydig.csv',
 'theimagingworld.csv',
 'johnlewis.csv',
 'keh.csv',
 'mikescamera.csv',
 'park.csv']

In [13]:
data1=pd.read_csv('buydig.csv')
data1=data1.rename(index=str, columns={"id": "identifier"})
data2=pd.read_csv('jessops.csv')
data2=data2.rename(index=str, columns={"id": "identifier"})
data3=pd.read_csv('johnlewis.csv')
data3=data3.rename(index=str, columns={"id": "identifier"})
data1=data1.append(data2)
data1=data1.append(data3)

In [14]:
i=0
n=len(array_list)
while(i<n):
    if(i==3 | i==16| i==18 | i==13):
        i=i+1
    else:
        df=pd.read_csv(array_list[i])
        data1=data1.append(df)
        i=i+1

In [15]:
data1=data1.reset_index().drop('index',axis=1)
data1.shape

(1267619, 396)

In [16]:
data1.head()

,0mp,10,1037k,1080p,12mp,14,14mp,15,150,150mm,...,wifi,with,wr,x70,yellow,zink,zoom,zs70,zuiko,zx5
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
data1.tail()

,0mp,10,1037k,1080p,12mp,14,14mp,15,150,150mm,...,wifi,with,wr,x70,yellow,zink,zoom,zs70,zuiko,zx5
1267614,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1267615,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1267616,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1267617,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1267618,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
cam_gs=list(cam_gold_standard['page_id'].unique())
print(len(cam_gs))
info_pages=data1[data1['identifier'].isin(cam_gs)]
print(info_pages.shape)

120
(120, 396)


In [19]:
info_pages.head()

,0mp,10,1037k,1080p,12mp,14,14mp,15,150,150mm,...,wifi,with,wr,x70,yellow,zink,zoom,zs70,zuiko,zx5
132,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
164,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
918,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1034,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1474,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
info_pages.columns.values

array(['0mp', '10', '1037k', '1080p', '12mp', '14', '14mp', '15', '150',
       '150mm', '16', '16519247', '16519314', '16gb', '170', '18', '18mp',
       '1mp', '20', '200mm', '23', '24', '24mp', '26mm', '27709', '27mm',
       '29mp', '2mp', '2nd', '2x', '2x3', '30', '300', '300mm', '30fps',
       '30mm', '30x', '32gb', '360', '40', '42', '42mm', '45', '45mm',
       '48gb', '4k', '4x', '50mm', '50mmf2', '5100l', '55', '55mm', '58mm',
       '5mp', '5x', '600', '60x', '633x', '64g', '64gb', '6k', '70',
       '720p', '80', '8mp', '8pc', '8x', '921', 'PRED', 'PRED_PROBA',
       'Unnamed: 0', 'a5100', 'a68', 'accessories', 'accessory', 'action',
       'activeon', 'alpha', 'analog', 'and', 'angle', 'angle_zoom',
       'as300', 'automat', 'availability', 'az401', 'backed', 'backpack',
       'bag', 'batteries', 'battery', 'bell', 'bestRating', 'bk', 'black',
       'blk', 'bn1', 'body', 'bottom', 'brand', 'bridge', 'bubble',
       'bundle', 'buoy', 'by', 'cam', 'camcorder', 'camera'

In [21]:
page_df=pd.merge(cam_gold_standard, info_pages[['identifier','name','brand','price']], how='left', left_on='page_id', right_on='identifier')
page_df=page_df[['page_id','catalog_id','label','name','brand','price']]
page_df.columns=['page_id','catalog_id','label','page_name','page_brand','page_price']
page_df.head()

,page_id,catalog_id,label,page_name,page_brand,page_price
0,argos-23426c3985d6e24911336f3d8106409b,camera_001,0,Fujifilm X T2 Mirrorless Camera Body Black,//www.argos.co.uk/search/Fuji,1434
1,argos-23426c3985d6e24911336f3d8106409b,camera_002,0,Fujifilm X T2 Mirrorless Camera Body Black,//www.argos.co.uk/search/Fuji,1434
2,argos-23426c3985d6e24911336f3d8106409b,camera_003,0,Fujifilm X T2 Mirrorless Camera Body Black,//www.argos.co.uk/search/Fuji,1434
3,argos-23426c3985d6e24911336f3d8106409b,camera_004,0,Fujifilm X T2 Mirrorless Camera Body Black,//www.argos.co.uk/search/Fuji,1434
4,argos-23426c3985d6e24911336f3d8106409b,camera_005,0,Fujifilm X T2 Mirrorless Camera Body Black,//www.argos.co.uk/search/Fuji,1434


In [22]:
page_df.shape #page id and catalog id matchings and 

(6000, 6)

In [23]:
bow_product_catalog.shape #catalog information

(50, 24)

In [24]:
bow_product_catalog.columns

Index([u'id', u'id_2', u'name', u'brand', u'type', u'color', u'itemCondition',
       u'availability', u'priceCurrency', u'price', u'megapixels',
       u'optical_zoom', u'video_resolution', u'iso_range', u'unit', u'width',
       u'height', u'depth', u'unit2', u'value', u'model', u'has_wifi',
       u'has_bluetooth', u'battery'],
      dtype='object')

In [25]:
all_df=pd.merge(page_df, bow_product_catalog[['id_2','name','brand','price']], how='left', left_on='catalog_id', right_on='id_2')
all_df.shape

(6000, 10)

In [26]:
all_df.head()

,page_id,catalog_id,label,page_name,page_brand,page_price,id_2,name,brand,price
0,argos-23426c3985d6e24911336f3d8106409b,camera_001,0,Fujifilm X T2 Mirrorless Camera Body Black,//www.argos.co.uk/search/Fuji,1434,camera_001,Nikon D3400 Digital SLR Camera with 24.2 Megap...,Nikon,569.99
1,argos-23426c3985d6e24911336f3d8106409b,camera_002,0,Fujifilm X T2 Mirrorless Camera Body Black,//www.argos.co.uk/search/Fuji,1434,camera_002,Canon EOS Rebel T6 Digital SLR Camera with 18-...,Canon,449.99
2,argos-23426c3985d6e24911336f3d8106409b,camera_003,0,Fujifilm X T2 Mirrorless Camera Body Black,//www.argos.co.uk/search/Fuji,1434,camera_003,Sony Alpha a68 Digital SLR Camera with DT 18-5...,Sony,698
3,argos-23426c3985d6e24911336f3d8106409b,camera_004,0,Fujifilm X T2 Mirrorless Camera Body Black,//www.argos.co.uk/search/Fuji,1434,camera_004,Pentax K-1 Mark II Digital SLR Camera Body,Pentax,"1,996.95"
4,argos-23426c3985d6e24911336f3d8106409b,camera_005,0,Fujifilm X T2 Mirrorless Camera Body Black,//www.argos.co.uk/search/Fuji,1434,camera_005,Panasonic - LUMIX DC-ZS70 20.3-Megapixel Digit...,Panasonic,449.99


# 1. ONLY NAMES

## 1.1. Bag of Words (1,0) with only names

In [27]:
df_only_names=all_df[['page_name','name']]
df_only_names.head()

,page_name,name
0,Fujifilm X T2 Mirrorless Camera Body Black,Nikon D3400 Digital SLR Camera with 24.2 Megap...
1,Fujifilm X T2 Mirrorless Camera Body Black,Canon EOS Rebel T6 Digital SLR Camera with 18-...
2,Fujifilm X T2 Mirrorless Camera Body Black,Sony Alpha a68 Digital SLR Camera with DT 18-5...
3,Fujifilm X T2 Mirrorless Camera Body Black,Pentax K-1 Mark II Digital SLR Camera Body
4,Fujifilm X T2 Mirrorless Camera Body Black,Panasonic - LUMIX DC-ZS70 20.3-Megapixel Digit...


In [29]:
df_try=pd.DataFrame([df_only_names['page_name'],df_only_names['name']])

In [30]:
df_try.head()

,0,1,2,3,4,5,6,7,8,9,...,5990,5991,5992,5993,5994,5995,5996,5997,5998,5999
page_name,Fujifilm X T2 Mirrorless Camera Body Black,Fujifilm X T2 Mirrorless Camera Body Black,Fujifilm X T2 Mirrorless Camera Body Black,Fujifilm X T2 Mirrorless Camera Body Black,Fujifilm X T2 Mirrorless Camera Body Black,Fujifilm X T2 Mirrorless Camera Body Black,Fujifilm X T2 Mirrorless Camera Body Black,Fujifilm X T2 Mirrorless Camera Body Black,Fujifilm X T2 Mirrorless Camera Body Black,Fujifilm X T2 Mirrorless Camera Body Black,...,Ricoh WG-M2 Action Camera (Silver),Ricoh WG-M2 Action Camera (Silver),Ricoh WG-M2 Action Camera (Silver),Ricoh WG-M2 Action Camera (Silver),Ricoh WG-M2 Action Camera (Silver),Ricoh WG-M2 Action Camera (Silver),Ricoh WG-M2 Action Camera (Silver),Ricoh WG-M2 Action Camera (Silver),Ricoh WG-M2 Action Camera (Silver),Ricoh WG-M2 Action Camera (Silver)
name,Nikon D3400 Digital SLR Camera with 24.2 Megap...,Canon EOS Rebel T6 Digital SLR Camera with 18-...,Sony Alpha a68 Digital SLR Camera with DT 18-5...,Pentax K-1 Mark II Digital SLR Camera Body,Panasonic - LUMIX DC-ZS70 20.3-Megapixel Digit...,Olympus OM-D E-M10 Mark II Digital Camera w 14...,Fujifilm X70 Digital Camera (Black),Sony - AS300 Waterproof Action Camera - White,Nikon - KeyMission 360 Degree Waterproof Actio...,GoPro - HERO5 Session 4K Action Camera - Black,...,CANON EOS M50 Mirrorless Camera with EF-M 15-4...,NIKON 1 J5 Mirrorless Camera with 10-30 mm f/3...,Impossible I-1 Camera,SAMSUNG NX30 Mirrorless Digital Camera with 18...,NIKON COOLPIX P900 Bridge Camera - Black,CANON PowerShot SX540 HS Bridge Camera - Black,SONY Cyber-shot DSCH300B Bridge Camera,KODAK PIXPRO AZ401 Bridge Camera - Black,PANASONIC Lumix DC-FZ82EB-K Bridge Camera - Black,Olympus - TG-Tracker 4K Waterproof Action Came...


- Take bags of words for each page_name and name pair, turn it into bags of words and calculate cosine similarity

### 1.1.1. Cosine Similarity

- Cosine Similarity

In [35]:
i=0
n=df_try.shape[1]
cosine=[]
while(i<n):
    vectorizer = CountVectorizer()
    matrix=vectorizer.fit_transform(df_try[i]).toarray()
    vocab=vectorizer.vocabulary_ 
    bags_words=pd.DataFrame(matrix, columns=sorted(vocab))
    cosine.append(pairwise.cosine_similarity(X=bags_words.iloc[[0]] , Y=bags_words.iloc[[1]])[0][0])
    i=i+1

In [27]:
len(cosine)

6000

In [28]:
df_only_names['label']=all_df['label']
df_only_names['cosine_similarity']=cosine

/Users/bengikoseoglu/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/bengikoseoglu/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


- Find the threshold and calculate the results

In [29]:
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss
from sklearn.metrics import roc_auc_score

def confusion_matrix_report(y_true, y_pred):
    cm, labels = confusion_matrix(y_true, y_pred), unique_labels(y_true, y_pred)
    column_width = max([len(str(x)) for x in labels] + [5])  # 5 is value length
    report = " " * column_width + " " + "{:_^{}}".format("Prediction", column_width * len(labels))+ "\n"
    report += " " * column_width + " ".join(["{:>{}}".format(label, column_width) for label in labels]) + "\n"
    for i, label1 in enumerate(labels):
        report += "{:>{}}".format(label1, column_width) + " ".join(["{:{}d}".format(cm[i, j], column_width) for j in range(len(labels))]) + "\n"
    return report
    
def analytics(y_true, y_pred):#target of the test data #predictions as 0,1 #model (knnclassifier) #data_test
    print("Confusion Matrix:")
    print(confusion_matrix_report(y_true, y_pred))
    print("Accuracy Score:")
    print(accuracy_score(y_true, y_pred))
    print()
    print("Classification Report:")
    print(classification_report(y_true, y_pred))
    #print("AUC Score:")
    #print(AUC(target_test, model, data_test))

In [30]:
#measure the accruacy for different levels
i=0
n=df_only_names.shape[0]
ls=[0.05,0.1,0.15,0.2,0.25,0.30,0.35,0.40,0.45,0.50,0.55,0.60,0.65,0.70,0.75,0.80,0.85,0.9,0.95]
while(i<len(ls)):
    k=0
    score_cosine=[]
    while(k<n):
        if(df_only_names.loc[k]['cosine_similarity']>ls[i]):
            score_cosine.append(1)
        else:
            score_cosine.append(0)
        k=k+1
    print('Threshold: {}'.format(ls[i]))
    print(analytics(df_only_names['label'], score_cosine))
    print('*********************************')
    i=i+1

Threshold: 0.05
Confusion Matrix:
      Prediction
         0     1
    0 1254  4628
    1    0   118

Accuracy Score:
0.228666666667
()
Classification Report:
             precision    recall  f1-score   support

          0       1.00      0.21      0.35      5882
          1       0.02      1.00      0.05       118

avg / total       0.98      0.23      0.35      6000

None
*********************************
Threshold: 0.1
Confusion Matrix:
      Prediction
         0     1
    0 2220  3662
    1    0   118

Accuracy Score:
0.389666666667
()
Classification Report:
             precision    recall  f1-score   support

          0       1.00      0.38      0.55      5882
          1       0.03      1.00      0.06       118

avg / total       0.98      0.39      0.54      6000

None
*********************************
Threshold: 0.15
Confusion Matrix:
      Prediction
         0     1
    0 3657  2225
    1    0   118

Accuracy Score:
0.629166666667
()
Classification Report:
             

- According to the above thresholds and results, the threshold should be 0.55 due to the fact that it has the highest f1 score for 1 

In [31]:
score_cosine=[]
k=0
while(k<n):
    if(df_only_names.loc[k]['cosine_similarity']>0.55):
            score_cosine.append(1)
    else:
            score_cosine.append(0)
    k=k+1
print(analytics(df_only_names['label'], score_cosine))

Confusion Matrix:
      Prediction
         0     1
    0 5863    19
    1   31    87

Accuracy Score:
0.991666666667
()
Classification Report:
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      5882
          1       0.82      0.74      0.78       118

avg / total       0.99      0.99      0.99      6000

None


### 1.1.2. Jaccard Similarity

In [32]:
# inspired by http://billchambers.me/tutorials/2014/12/22/cosine-similarity-explained-in-python.html
import string
import math
from __future__ import division
def jaccard_similarity(query, document):
    intersection = set(query).intersection(set(document))
    union = set(query).union(set(document))
    return len(intersection)/len(union)

In [33]:
list_extracted=list(df_only_names['page_name'])
list_gs=list(df_only_names['name'])
tokenize = lambda doc: doc.lower().split(" ")
tokenized_extr = [tokenize(d) for d in list_extracted] # tokenized docs
tokenized_gs = [tokenize(d) for d in list_gs] # tokenized docs

In [34]:
# measure the jaccard similarity
#.as_matrix
i=0
n=df_only_names.shape[0]
jaccard=[]
while(i<n):
    jaccard.append(jaccard_similarity(tokenized_extr[i],tokenized_gs[i]))
    i=i+1

In [35]:
len(jaccard)

6000

In [36]:
df_only_names['jaccard_similarity']=jaccard

/Users/bengikoseoglu/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [37]:
#measure the accruacy for different levels
i=0
n=df_only_names.shape[0]
ls=[0.05,0.1,0.15,0.2,0.25,0.30,0.35,0.40,0.45,0.50,0.55,0.60,0.65,0.70,0.75,0.80,0.85,0.9,0.95]
while(i<len(ls)):
    k=0
    score_cosine=[]
    while(k<n):
        if(df_only_names.loc[k]['jaccard_similarity']>ls[i]):
            score_cosine.append(1)
        else:
            score_cosine.append(0)
        k=k+1
    print('Threshold: {}'.format(ls[i]))
    print(analytics(df_only_names['label'], score_cosine))
    print('*********************************')
    i=i+1

Threshold: 0.05
Confusion Matrix:
      Prediction
         0     1
    0 2407  3475
    1    0   118

Accuracy Score:
0.420833333333
()
Classification Report:
             precision    recall  f1-score   support

          0       1.00      0.41      0.58      5882
          1       0.03      1.00      0.06       118

avg / total       0.98      0.42      0.57      6000

None
*********************************
Threshold: 0.1
Confusion Matrix:
      Prediction
         0     1
    0 4435  1447
    1    1   117

Accuracy Score:
0.758666666667
()
Classification Report:
             precision    recall  f1-score   support

          0       1.00      0.75      0.86      5882
          1       0.07      0.99      0.14       118

avg / total       0.98      0.76      0.85      6000

None
*********************************
Threshold: 0.15
Confusion Matrix:
      Prediction
         0     1
    0 5260   622
    1    3   115

Accuracy Score:
0.895833333333
()
Classification Report:
             

- According to the above thresholds and results, the threshold should be 0.3 due to the fact that it has the highest f1 score for 1 

In [38]:
score_cosine=[]
k=0
while(k<n):
    if(df_only_names.loc[k]['jaccard_similarity']>0.3):
            score_cosine.append(1)
    else:
            score_cosine.append(0)
    k=k+1
print(analytics(df_only_names['label'], score_cosine))

Confusion Matrix:
      Prediction
         0     1
    0 5835    47
    1   30    88

Accuracy Score:
0.987166666667
()
Classification Report:
             precision    recall  f1-score   support

          0       0.99      0.99      0.99      5882
          1       0.65      0.75      0.70       118

avg / total       0.99      0.99      0.99      6000

None


In [39]:
df_only_names.head()

,page_name,name,label,cosine_similarity,jaccard_similarity
0,Fujifilm X T2 Mirrorless Camera Body Black,Nikon D3400 Digital SLR Camera with 24.2 Megap...,0,0.096225,0.052632
1,Fujifilm X T2 Mirrorless Camera Body Black,Canon EOS Rebel T6 Digital SLR Camera with 18-...,0,0.065372,0.026316
2,Fujifilm X T2 Mirrorless Camera Body Black,Sony Alpha a68 Digital SLR Camera with DT 18-5...,0,0.113228,0.052632
3,Fujifilm X T2 Mirrorless Camera Body Black,Pentax K-1 Mark II Digital SLR Camera Body,0,0.308607,0.153846
4,Fujifilm X T2 Mirrorless Camera Body Black,Panasonic - LUMIX DC-ZS70 20.3-Megapixel Digit...,0,0.272166,0.153846


## 1.2. TFIDF Vectors

### 1.2.1. Cosine Similarity

In [40]:
df_try.head()

,0,1,2,3,4,5,6,7,8,9,...,5990,5991,5992,5993,5994,5995,5996,5997,5998,5999
page_name,Fujifilm X T2 Mirrorless Camera Body Black,Fujifilm X T2 Mirrorless Camera Body Black,Fujifilm X T2 Mirrorless Camera Body Black,Fujifilm X T2 Mirrorless Camera Body Black,Fujifilm X T2 Mirrorless Camera Body Black,Fujifilm X T2 Mirrorless Camera Body Black,Fujifilm X T2 Mirrorless Camera Body Black,Fujifilm X T2 Mirrorless Camera Body Black,Fujifilm X T2 Mirrorless Camera Body Black,Fujifilm X T2 Mirrorless Camera Body Black,...,Ricoh WG-M2 Action Camera (Silver),Ricoh WG-M2 Action Camera (Silver),Ricoh WG-M2 Action Camera (Silver),Ricoh WG-M2 Action Camera (Silver),Ricoh WG-M2 Action Camera (Silver),Ricoh WG-M2 Action Camera (Silver),Ricoh WG-M2 Action Camera (Silver),Ricoh WG-M2 Action Camera (Silver),Ricoh WG-M2 Action Camera (Silver),Ricoh WG-M2 Action Camera (Silver)
name,Nikon D3400 Digital SLR Camera with 24.2 Megap...,Canon EOS Rebel T6 Digital SLR Camera with 18-...,Sony Alpha a68 Digital SLR Camera with DT 18-5...,Pentax K-1 Mark II Digital SLR Camera Body,Panasonic - LUMIX DC-ZS70 20.3-Megapixel Digit...,Olympus OM-D E-M10 Mark II Digital Camera w 14...,Fujifilm X70 Digital Camera (Black),Sony - AS300 Waterproof Action Camera - White,Nikon - KeyMission 360 Degree Waterproof Actio...,GoPro - HERO5 Session 4K Action Camera - Black,...,CANON EOS M50 Mirrorless Camera with EF-M 15-4...,NIKON 1 J5 Mirrorless Camera with 10-30 mm f/3...,Impossible I-1 Camera,SAMSUNG NX30 Mirrorless Digital Camera with 18...,NIKON COOLPIX P900 Bridge Camera - Black,CANON PowerShot SX540 HS Bridge Camera - Black,SONY Cyber-shot DSCH300B Bridge Camera,KODAK PIXPRO AZ401 Bridge Camera - Black,PANASONIC Lumix DC-FZ82EB-K Bridge Camera - Black,Olympus - TG-Tracker 4K Waterproof Action Came...


In [41]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
i=0
n=df_try.shape[1]
cosine=[]
while(i<n):
    vectorizer = CountVectorizer()
    matrix=vectorizer.fit_transform(df_try[i]).toarray()
    tfidf_r = tfidf_transformer.fit_transform(matrix)
    vocab=vectorizer.vocabulary_ 
    bags_words=pd.DataFrame([tfidf_r.toarray()[0],tfidf_r.toarray()[1]], columns=sorted(vocab))
    cosine.append(pairwise.cosine_similarity(X=bags_words.iloc[[0]] , Y=bags_words.iloc[[1]])[0][0])
    i=i+1

In [42]:
len(cosine)

6000

In [43]:
df_only_names['cosine_similarity_tfidf']=cosine

/Users/bengikoseoglu/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [44]:
df_only_names.head()

,page_name,name,label,cosine_similarity,jaccard_similarity,cosine_similarity_tfidf
0,Fujifilm X T2 Mirrorless Camera Body Black,Nikon D3400 Digital SLR Camera with 24.2 Megap...,0,0.096225,0.052632,0.051563
1,Fujifilm X T2 Mirrorless Camera Body Black,Canon EOS Rebel T6 Digital SLR Camera with 18-...,0,0.065372,0.026316,0.034767
2,Fujifilm X T2 Mirrorless Camera Body Black,Sony Alpha a68 Digital SLR Camera with DT 18-5...,0,0.113228,0.052632,0.061005
3,Fujifilm X T2 Mirrorless Camera Body Black,Pentax K-1 Mark II Digital SLR Camera Body,0,0.308607,0.153846,0.184432
4,Fujifilm X T2 Mirrorless Camera Body Black,Panasonic - LUMIX DC-ZS70 20.3-Megapixel Digit...,0,0.272166,0.153846,0.159764


In [45]:
#measure the accruacy for different levels
i=0
n=df_only_names.shape[0]
ls=[0.05,0.1,0.15,0.2,0.25,0.30,0.35,0.40,0.45,0.50,0.55,0.60,0.65,0.70,0.75,0.80,0.85,0.9,0.95]
while(i<len(ls)):
    k=0
    score_cosine=[]
    while(k<n):
        if(df_only_names.loc[k]['cosine_similarity_tfidf']>ls[i]):
            score_cosine.append(1)
        else:
            score_cosine.append(0)
        k=k+1
    print('Threshold: {}'.format(ls[i]))
    print(analytics(df_only_names['label'], score_cosine))
    print('*********************************')
    i=i+1

Threshold: 0.05
Confusion Matrix:
      Prediction
         0     1
    0 2007  3875
    1    0   118

Accuracy Score:
0.354166666667
()
Classification Report:
             precision    recall  f1-score   support

          0       1.00      0.34      0.51      5882
          1       0.03      1.00      0.06       118

avg / total       0.98      0.35      0.50      6000

None
*********************************
Threshold: 0.1
Confusion Matrix:
      Prediction
         0     1
    0 4213  1669
    1    0   118

Accuracy Score:
0.721833333333
()
Classification Report:
             precision    recall  f1-score   support

          0       1.00      0.72      0.83      5882
          1       0.07      1.00      0.12       118

avg / total       0.98      0.72      0.82      6000

None
*********************************
Threshold: 0.15
Confusion Matrix:
      Prediction
         0     1
    0 5141   741
    1    1   117

Accuracy Score:
0.876333333333
()
Classification Report:
             

- According to the above thresholds and results, the threshold should be 0.4 due to the fact that it has the highest f1 score for 1 

In [46]:
score_cosine=[]
k=0
while(k<n):
    if(df_only_names.loc[k]['cosine_similarity_tfidf']>0.4):
            score_cosine.append(1)
    else:
            score_cosine.append(0)
    k=k+1
print(analytics(df_only_names['label'], score_cosine))

Confusion Matrix:
      Prediction
         0     1
    0 5865    17
    1   32    86

Accuracy Score:
0.991833333333
()
Classification Report:
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      5882
          1       0.83      0.73      0.78       118

avg / total       0.99      0.99      0.99      6000

None


# 2. Including Other Variables

- Previously we have only used names to calculate similairties between pages and categories, now we are going to use other vairblaes as well such as brand and price

## 2.1. Bag of Words

In [47]:
all_df2=pd.merge(page_df, bow_product_catalog[['id_2','name','brand','price']], how='left', left_on='catalog_id', right_on='id_2')
all_df2=all_df2.drop('id_2',axis=1)
all_df2.shape

(6000, 9)

In [48]:
# fill missing values in product pages
most_common_brand=all_df2['page_brand'].value_counts()[1]
most_common_price=all_df2['page_price'].value_counts()[0]
all_df2['page_brand'] = all_df2['page_brand'].replace(np.nan, most_common_brand, regex=True)
all_df2['page_price'] = all_df2['page_price'].replace(np.nan, most_common_brand, regex=True)
all_df2['page_brand'] = all_df2['page_brand'].replace(np.nan, most_common_brand, regex=True)

In [49]:
all_df2.head()

,page_id,catalog_id,label,page_name,page_brand,page_price,name,brand,price
0,argos-23426c3985d6e24911336f3d8106409b,camera_001,0,Fujifilm X T2 Mirrorless Camera Body Black,//www.argos.co.uk/search/Fuji,1434,Nikon D3400 Digital SLR Camera with 24.2 Megap...,Nikon,569.99
1,argos-23426c3985d6e24911336f3d8106409b,camera_002,0,Fujifilm X T2 Mirrorless Camera Body Black,//www.argos.co.uk/search/Fuji,1434,Canon EOS Rebel T6 Digital SLR Camera with 18-...,Canon,449.99
2,argos-23426c3985d6e24911336f3d8106409b,camera_003,0,Fujifilm X T2 Mirrorless Camera Body Black,//www.argos.co.uk/search/Fuji,1434,Sony Alpha a68 Digital SLR Camera with DT 18-5...,Sony,698
3,argos-23426c3985d6e24911336f3d8106409b,camera_004,0,Fujifilm X T2 Mirrorless Camera Body Black,//www.argos.co.uk/search/Fuji,1434,Pentax K-1 Mark II Digital SLR Camera Body,Pentax,"1,996.95"
4,argos-23426c3985d6e24911336f3d8106409b,camera_005,0,Fujifilm X T2 Mirrorless Camera Body Black,//www.argos.co.uk/search/Fuji,1434,Panasonic - LUMIX DC-ZS70 20.3-Megapixel Digit...,Panasonic,449.99


In [50]:
all_df2.loc[i]

page_id            argos-23426c3985d6e24911336f3d8106409b
catalog_id                                     camera_020
label                                                   0
page_name      Fujifilm X T2 Mirrorless Camera Body Black
page_brand                  //www.argos.co.uk/search/Fuji
page_price                                           1434
name          Lomo Instant Automat Glass Magellan (Black)
brand                                          Lomography
price                                                 189
Name: 19, dtype: object

In [51]:
all_df2.dtypes

page_id       object
catalog_id    object
label          int64
page_name     object
page_brand    object
page_price    object
name          object
brand         object
price         object
dtype: object

In [52]:
all_df2['brand'].unique()

array(['Nikon', 'Canon', 'Sony', 'Pentax', 'Panasonic', 'Olympus',
       'Fujifilm', 'GoPro', 'Polaroid', 'ACTIVEON', 'Mevo - Live', 'SJCAM',
       'Vivitar', 'Leica', 'Kodak', 'Lomography', 'Bell  Howell', 'JVC',
       'SONY', 'Ricoh', 'OLYMPUS', 'KODAK', 'PANASONIC', 'FUJIFILM',
       'CANON', 'NIKON', 'Impossible', 'SAMSUNG'], dtype=object)

### 2.1.1. Cosine Similarity

In [53]:
page_t=list(all_df2['page_name']+' '+all_df2['page_brand'].astype(str)+' '+all_df2['page_price'].astype(str))
cat_t=list(all_df2['name']+' '+all_df2['brand'].astype(str)+' '+all_df2['price'].astype(str))
cat_t2=[x.decode('utf-8','ignore').encode("utf-8") for x in cat_t]
page_t2=[x.decode('utf-8','ignore').encode("utf-8") for x in page_t]
df_try=pd.DataFrame([page_t2,cat_t2])
df_try.head()

,0,1,2,3,4,5,6,7,8,9,...,5990,5991,5992,5993,5994,5995,5996,5997,5998,5999
0,Fujifilm X T2 Mirrorless Camera Body Black //w...,Fujifilm X T2 Mirrorless Camera Body Black //w...,Fujifilm X T2 Mirrorless Camera Body Black //w...,Fujifilm X T2 Mirrorless Camera Body Black //w...,Fujifilm X T2 Mirrorless Camera Body Black //w...,Fujifilm X T2 Mirrorless Camera Body Black //w...,Fujifilm X T2 Mirrorless Camera Body Black //w...,Fujifilm X T2 Mirrorless Camera Body Black //w...,Fujifilm X T2 Mirrorless Camera Body Black //w...,Fujifilm X T2 Mirrorless Camera Body Black //w...,...,Ricoh WG-M2 Action Camera (Silver) Ricoh 149.0,Ricoh WG-M2 Action Camera (Silver) Ricoh 149.0,Ricoh WG-M2 Action Camera (Silver) Ricoh 149.0,Ricoh WG-M2 Action Camera (Silver) Ricoh 149.0,Ricoh WG-M2 Action Camera (Silver) Ricoh 149.0,Ricoh WG-M2 Action Camera (Silver) Ricoh 149.0,Ricoh WG-M2 Action Camera (Silver) Ricoh 149.0,Ricoh WG-M2 Action Camera (Silver) Ricoh 149.0,Ricoh WG-M2 Action Camera (Silver) Ricoh 149.0,Ricoh WG-M2 Action Camera (Silver) Ricoh 149.0
1,Nikon D3400 Digital SLR Camera with 24.2 Megap...,Canon EOS Rebel T6 Digital SLR Camera with 18-...,Sony Alpha a68 Digital SLR Camera with DT 18-5...,Pentax K-1 Mark II Digital SLR Camera Body Pen...,Panasonic - LUMIX DC-ZS70 20.3-Megapixel Digit...,Olympus OM-D E-M10 Mark II Digital Camera w 14...,Fujifilm X70 Digital Camera (Black) Fujifilm 699,Sony - AS300 Waterproof Action Camera - White ...,Nikon - KeyMission 360 Degree Waterproof Actio...,GoPro - HERO5 Session 4K Action Camera - Black...,...,CANON EOS M50 Mirrorless Camera with EF-M 15-4...,NIKON 1 J5 Mirrorless Camera with 10-30 mm f/3...,Impossible I-1 Camera Impossible 149,SAMSUNG NX30 Mirrorless Digital Camera with 18...,NIKON COOLPIX P900 Bridge Camera - Black Nikon...,CANON PowerShot SX540 HS Bridge Camera - Black...,SONY Cyber-shot DSCH300B Bridge Camera SONY 139,KODAK PIXPRO AZ401 Bridge Camera - Black KODAK...,PANASONIC Lumix DC-FZ82EB-K Bridge Camera - Bl...,Olympus - TG-Tracker 4K Waterproof Action Came...


In [54]:
i=0
n=df_try.shape[1]
cosine=[]
while(i<n):
    vectorizer = CountVectorizer()
    matrix=vectorizer.fit_transform(df_try[i]).toarray()
    vocab=vectorizer.vocabulary_ 
    bags_words=pd.DataFrame(matrix, columns=sorted(vocab))
    cosine.append(pairwise.cosine_similarity(X=bags_words.iloc[[0]] , Y=bags_words.iloc[[1]])[0][0])
    i=i+1

In [55]:
df_only_names['cosine_similarity_other']=cosine

/Users/bengikoseoglu/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [56]:
#measure the accruacy for different levels
i=0
n=df_only_names.shape[0]
ls=[0.05,0.1,0.15,0.2,0.25,0.30,0.35,0.40,0.45,0.50,0.55,0.60,0.65,0.70,0.75,0.80,0.85,0.9,0.95]
while(i<len(ls)):
    k=0
    score_cosine=[]
    while(k<n):
        if(df_only_names.loc[k]['cosine_similarity_other']>ls[i]):
            score_cosine.append(1)
        else:
            score_cosine.append(0)
        k=k+1
    print('Threshold: {}'.format(ls[i]))
    print(analytics(df_only_names['label'], score_cosine))
    print('*********************************')
    i=i+1

Threshold: 0.05
Confusion Matrix:
      Prediction
         0     1
    0 1184  4698
    1    0   118

Accuracy Score:
0.217
()
Classification Report:
             precision    recall  f1-score   support

          0       1.00      0.20      0.34      5882
          1       0.02      1.00      0.05       118

avg / total       0.98      0.22      0.33      6000

None
*********************************
Threshold: 0.1
Confusion Matrix:
      Prediction
         0     1
    0 3213  2669
    1    0   118

Accuracy Score:
0.555166666667
()
Classification Report:
             precision    recall  f1-score   support

          0       1.00      0.55      0.71      5882
          1       0.04      1.00      0.08       118

avg / total       0.98      0.56      0.69      6000

None
*********************************
Threshold: 0.15
Confusion Matrix:
      Prediction
         0     1
    0 4300  1582
    1    0   118

Accuracy Score:
0.736333333333
()
Classification Report:
             precision

- According to the above thresholds and results, the threshold should be 0.50 due to the fact that it has the highest f1 score for 1 

In [57]:
score_cosine=[]
k=0
while(k<n):
    if(df_only_names.loc[k]['cosine_similarity_other']>0.55):
            score_cosine.append(1)
    else:
            score_cosine.append(0)
    k=k+1
print(analytics(df_only_names['label'], score_cosine))

Confusion Matrix:
      Prediction
         0     1
    0 5872    10
    1   33    85

Accuracy Score:
0.992833333333
()
Classification Report:
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      5882
          1       0.89      0.72      0.80       118

avg / total       0.99      0.99      0.99      6000

None


### 2.1.2. Jaccard Similarity

In [58]:
list_extracted=list(df_try.loc[0])
list_gs=list(df_try.loc[1])
tokenize = lambda doc: doc.lower().split(" ")
tokenized_extr = [tokenize(d) for d in list_extracted] # tokenized docs
tokenized_gs = [tokenize(d) for d in list_gs] # tokenized docs

In [59]:
# measure the jaccard similarity
#.as_matrix
i=0
n=df_only_names.shape[0]
jaccard=[]
while(i<n):
    jaccard.append(jaccard_similarity(tokenized_extr[i],tokenized_gs[i]))
    i=i+1

In [60]:
df_only_names['jaccard_similarity_other']=jaccard

/Users/bengikoseoglu/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [61]:
#measure the accruacy for different levels
i=0
n=df_only_names.shape[0]
ls=[0.05,0.1,0.15,0.2,0.25,0.30,0.35,0.40,0.45,0.50,0.55,0.60,0.65,0.70,0.75,0.80,0.85,0.9,0.95]
while(i<len(ls)):
    k=0
    score_cosine=[]
    while(k<n):
        if(df_only_names.loc[k]['jaccard_similarity_other']>ls[i]):
            score_cosine.append(1)
        else:
            score_cosine.append(0)
        k=k+1
    print('Threshold: {}'.format(ls[i]))
    print(analytics(df_only_names['label'], score_cosine))
    print('*********************************')
    i=i+1

Threshold: 0.05
Confusion Matrix:
      Prediction
         0     1
    0 2817  3065
    1    0   118

Accuracy Score:
0.489166666667
()
Classification Report:
             precision    recall  f1-score   support

          0       1.00      0.48      0.65      5882
          1       0.04      1.00      0.07       118

avg / total       0.98      0.49      0.64      6000

None
*********************************
Threshold: 0.1
Confusion Matrix:
      Prediction
         0     1
    0 4747  1135
    1    1   117

Accuracy Score:
0.810666666667
()
Classification Report:
             precision    recall  f1-score   support

          0       1.00      0.81      0.89      5882
          1       0.09      0.99      0.17       118

avg / total       0.98      0.81      0.88      6000

None
*********************************
Threshold: 0.15
Confusion Matrix:
      Prediction
         0     1
    0 5460   422
    1    5   113

Accuracy Score:
0.928833333333
()
Classification Report:
             

/Users/bengikoseoglu/anaconda2/lib/python2.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


- According to the above thresholds and results, the threshold should be 0.30 due to the fact that it has the highest f1 score for 1 

In [62]:
score_cosine=[]
k=0
while(k<n):
    if(df_only_names.loc[k]['jaccard_similarity_other']>0.3):
            score_cosine.append(1)
    else:
            score_cosine.append(0)
    k=k+1
print(analytics(df_only_names['label'], score_cosine))

Confusion Matrix:
      Prediction
         0     1
    0 5860    22
    1   41    77

Accuracy Score:
0.9895
()
Classification Report:
             precision    recall  f1-score   support

          0       0.99      1.00      0.99      5882
          1       0.78      0.65      0.71       118

avg / total       0.99      0.99      0.99      6000

None


## 2.2. TFIDF Vectors

### 2.2.1. Cosine Similarity

In [63]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
i=0
n=df_try.shape[1]
cosine=[]
while(i<n):
    vectorizer = CountVectorizer()
    matrix=vectorizer.fit_transform(df_try[i]).toarray()
    tfidf_r = tfidf_transformer.fit_transform(matrix)
    vocab=vectorizer.vocabulary_ 
    bags_words=pd.DataFrame([tfidf_r.toarray()[0],tfidf_r.toarray()[1]], columns=sorted(vocab))
    cosine.append(pairwise.cosine_similarity(X=bags_words.iloc[[0]] , Y=bags_words.iloc[[1]])[0][0])
    i=i+1

In [64]:
df_only_names['cosine_similarity_tfidf_other']=cosine

/Users/bengikoseoglu/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [65]:
df_only_names.head()

,page_name,name,label,cosine_similarity,jaccard_similarity,cosine_similarity_tfidf,cosine_similarity_other,jaccard_similarity_other,cosine_similarity_tfidf_other
0,Fujifilm X T2 Mirrorless Camera Body Black,Nikon D3400 Digital SLR Camera with 24.2 Megap...,0,0.096225,0.052632,0.051563,0.057831,0.045455,0.030175
1,Fujifilm X T2 Mirrorless Camera Body Black,Canon EOS Rebel T6 Digital SLR Camera with 18-...,0,0.065372,0.026316,0.034767,0.041812,0.024390,0.021703
2,Fujifilm X T2 Mirrorless Camera Body Black,Sony Alpha a68 Digital SLR Camera with DT 18-5...,0,0.113228,0.052632,0.061005,0.067267,0.045455,0.035235
3,Fujifilm X T2 Mirrorless Camera Body Black,Pentax K-1 Mark II Digital SLR Camera Body,0,0.308607,0.153846,0.184432,0.160128,0.125000,0.088030
4,Fujifilm X T2 Mirrorless Camera Body Black,Panasonic - LUMIX DC-ZS70 20.3-Megapixel Digit...,0,0.272166,0.153846,0.159764,0.148250,0.125000,0.080983


In [66]:
#measure the accruacy for different levels
i=0
n=df_only_names.shape[0]
ls=[0.05,0.1,0.15,0.2,0.25,0.30,0.35,0.40,0.45,0.50,0.55,0.60,0.65,0.70,0.75,0.80,0.85,0.9,0.95]
while(i<len(ls)):
    k=0
    score_cosine=[]
    while(k<n):
        if(df_only_names.loc[k]['cosine_similarity_tfidf_other']>ls[i]):
            score_cosine.append(1)
        else:
            score_cosine.append(0)
        k=k+1
    print('Threshold: {}'.format(ls[i]))
    print(analytics(df_only_names['label'], score_cosine))
    print('*********************************')
    i=i+1

Threshold: 0.05
Confusion Matrix:
      Prediction
         0     1
    0 2991  2891
    1    0   118

Accuracy Score:
0.518166666667
()
Classification Report:
             precision    recall  f1-score   support

          0       1.00      0.51      0.67      5882
          1       0.04      1.00      0.08       118

avg / total       0.98      0.52      0.66      6000

None
*********************************
Threshold: 0.1
Confusion Matrix:
      Prediction
         0     1
    0 4777  1105
    1    0   118

Accuracy Score:
0.815833333333
()
Classification Report:
             precision    recall  f1-score   support

          0       1.00      0.81      0.90      5882
          1       0.10      1.00      0.18       118

avg / total       0.98      0.82      0.88      6000

None
*********************************
Threshold: 0.15
Confusion Matrix:
      Prediction
         0     1
    0 5429   453
    1    1   117

Accuracy Score:
0.924333333333
()
Classification Report:
             

- According to the above thresholds and results, the threshold should be 0.4 due to the fact that it has the highest f1 score for 1 

In [67]:
score_cosine=[]
k=0
while(k<n):
    if(df_only_names.loc[k]['cosine_similarity_tfidf_other']>0.4):
            score_cosine.append(1)
    else:
            score_cosine.append(0)
    k=k+1
print(analytics(df_only_names['label'], score_cosine))

Confusion Matrix:
      Prediction
         0     1
    0 5876     6
    1   37    81

Accuracy Score:
0.992833333333
()
Classification Report:
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      5882
          1       0.93      0.69      0.79       118

avg / total       0.99      0.99      0.99      6000

None


# 3. HTML

## 3.1. Bag of words (1,0)

### 3.1.1. Cosine Similariy

- Adding html pages as tokens

In [68]:
os.chdir('/Users/bengikoseoglu/Documents/Masters/Semester2/TeamProject/Product_Classification/Ersejda/options/cameras/cameras_GS_ html')

In [69]:
all_df=pd.merge(page_df[['page_id','catalog_id','label']], bow_product_catalog, how='left', left_on='catalog_id', right_on='id_2')
all_df=all_df.drop(['id','id_2'],axis=1)
all_df.shape

(6000, 25)

In [70]:
all_df.head()

,page_id,catalog_id,label,name,brand,type,color,itemCondition,availability,priceCurrency,...,unit,width,height,depth,unit2,value,model,has_wifi,has_bluetooth,battery
0,argos-23426c3985d6e24911336f3d8106409b,camera_001,0,Nikon D3400 Digital SLR Camera with 24.2 Megap...,Nikon,DSLR,Black,NewCondition,InStock,USD,...,Inches,6.5,5.32,11.02,lbs,4.03,D3400,Yes,yes,EN-EL14A r
1,argos-23426c3985d6e24911336f3d8106409b,camera_002,0,Canon EOS Rebel T6 Digital SLR Camera with 18-...,Canon,DSLR,Black,NewCondition,InStock,USD,...,inches,5.1,4,3.1,lbs,15.52,1159C003,yes,yes,1 x Canon
2,argos-23426c3985d6e24911336f3d8106409b,camera_003,0,Sony Alpha a68 Digital SLR Camera with DT 18-5...,Sony,DSLR,Black,NewCondition,InStock,USD,...,in,5.61,4.1,3.26,g,696,a68,no,no,NP-FM500H
3,argos-23426c3985d6e24911336f3d8106409b,camera_004,0,Pentax K-1 Mark II Digital SLR Camera Body,Pentax,DSLR,Black,NewCondition,InStock,USD,...,In,5.4,4.3,3.4,g,925,K-1 Mark II,yes,no,D-LI90 Rec
4,argos-23426c3985d6e24911336f3d8106409b,camera_005,0,Panasonic - LUMIX DC-ZS70 20.3-Megapixel Digit...,Panasonic,DSLR,Black,NewCondition,InStock,USD,...,inches,4.41,2.65,1.62,pounds,0.71,DC-ZS70K,yes,no,1 x Li-ion


Steps:
    1) Convert every column into string
    2) Noise Removal = removing special characters, that is still present in the dataset
    3) Tokenazation = seperate by the spaces
    4) Eliminating words that have more letters than 45, because the longest word in vocablary is  pneumonoultramicroscopicsilicovolcanoconiosis and it has 45 characters

In [71]:
#Convert every column to string
i=0
all_columns=['name',
 'brand',
 'type',
 'color',
 'itemCondition',
 'availability',
 'priceCurrency',
 'price',
 'megapixels',
 'optical_zoom',
 'video_resolution',
 'iso_range',
 'unit',
 'width',
 'height',
 'depth',
 'unit2',
 'value',
 'model',
 'has_wifi',
 'has_bluetooth',
 'battery']
while(i<len(all_columns)):
    all_df[all_columns[i]]= all_df[all_columns[i]].astype(str)
    i=i+1

In [72]:
# for noise removal
import re, string, unicodedata
import contractions
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)
def denoise_text(text):
    text = remove_between_square_brackets(text)
    return text
def replace_contractions(text):
    """Replace contractions in string of text"""
    return contractions.fix(text)

In [73]:
all_df.shape

(6000, 25)

In [74]:
import codecs
import unicodedata
from bs4 import BeautifulSoup
i=0
keys=list(all_df['page_id'])
cols=['name','brand','type','color','itemCondition','availability','priceCurrency','price','megapixels','optical_zoom','video_resolution',
     'iso_range','unit','width','height','depth','unit2','value','model','has_wifi','has_bluetooth','battery']
cosine=[]
n=len(keys)
while(i<n):

    #get the corresponding html and parse it
    thing=(keys[i].encode('ascii','ignore'))+'.html'
    f=codecs.open(thing, 'r')
    text1=f.read()
    cleantext = BeautifulSoup(text1, "lxml").text
    as_str=cleantext.encode('ascii','ignore')
    clean=as_str.translate(None, '\r\n\t\,)};{$.|:(#"[]/><+=')
    sample = denoise_text(clean)
    sample=replace_contractions(sample)

    #create one big string from the columns in the catalog
    ii=0
    a=all_df.loc[i]['name']
    while(ii<len(cols)):
        a=a+' '+all_df.loc[i][cols[ii]]
        ii=ii+1

    #create a dataframe to hold the strings
    df_try=pd.DataFrame([a,sample])

    #turn them into bag of words
    vectorizer = CountVectorizer()
    matrix=vectorizer.fit_transform(df_try[0]).toarray()
    vocab=vectorizer.vocabulary_
    bags_words=pd.DataFrame(matrix, columns=sorted(vocab))

    #eliminate tokens that are less than 46 length
    columns=list(bags_words.columns.values)
    n1=len(columns)
    vocab2=[]
    for k in range(0, n1):
        if(len(columns[k])<46):
            vocab2.append(columns[k])
    
    bags_words2=bags_words[vocab2]
    cosine.append(pairwise.cosine_similarity(X=bags_words2.iloc[[0]] , Y=bags_words2.iloc[[1]])[0][0])
    i=i+1

In [75]:
len(cosine)

6000

In [76]:
max(cosine)

0.51468019632677509

In [77]:
all_df.head()

,page_id,catalog_id,label,name,brand,type,color,itemCondition,availability,priceCurrency,...,unit,width,height,depth,unit2,value,model,has_wifi,has_bluetooth,battery
0,argos-23426c3985d6e24911336f3d8106409b,camera_001,0,Nikon D3400 Digital SLR Camera with 24.2 Megap...,Nikon,DSLR,Black,NewCondition,InStock,USD,...,Inches,6.5,5.32,11.02,lbs,4.03,D3400,Yes,yes,EN-EL14A r
1,argos-23426c3985d6e24911336f3d8106409b,camera_002,0,Canon EOS Rebel T6 Digital SLR Camera with 18-...,Canon,DSLR,Black,NewCondition,InStock,USD,...,inches,5.1,4,3.1,lbs,15.52,1159C003,yes,yes,1 x Canon
2,argos-23426c3985d6e24911336f3d8106409b,camera_003,0,Sony Alpha a68 Digital SLR Camera with DT 18-5...,Sony,DSLR,Black,NewCondition,InStock,USD,...,in,5.61,4.1,3.26,g,696,a68,no,no,NP-FM500H
3,argos-23426c3985d6e24911336f3d8106409b,camera_004,0,Pentax K-1 Mark II Digital SLR Camera Body,Pentax,DSLR,Black,NewCondition,InStock,USD,...,In,5.4,4.3,3.4,g,925,K-1 Mark II,yes,no,D-LI90 Rec
4,argos-23426c3985d6e24911336f3d8106409b,camera_005,0,Panasonic - LUMIX DC-ZS70 20.3-Megapixel Digit...,Panasonic,DSLR,Black,NewCondition,InStock,USD,...,inches,4.41,2.65,1.62,pounds,0.71,DC-ZS70K,yes,no,1 x Li-ion


In [78]:
df_only_names['cosine_similarity_html']=cosine

In [79]:
#measure the accruacy for different levels
i=0
n=df_only_names.shape[0]
ls=[0.05,0.1,0.15,0.2,0.25,0.30,0.35,0.40,0.45,0.50,0.55,0.60,0.65,0.70,0.75,0.80,0.85,0.9,0.95]
while(i<len(ls)):
    k=0
    score_cosine=[]
    while(k<n):
        if(df_only_names.loc[k]['cosine_similarity_html']>ls[i]):
            score_cosine.append(1)
        else:
            score_cosine.append(0)
        k=k+1
    print('Threshold: {}'.format(ls[i]))
    print(analytics(df_only_names['label'], score_cosine))
    print('*********************************')
    i=i+1

Threshold: 0.05
Confusion Matrix:
      Prediction
         0     1
    0 2813  3069
    1    2   116

Accuracy Score:
0.488166666667
()
Classification Report:
             precision    recall  f1-score   support

          0       1.00      0.48      0.65      5882
          1       0.04      0.98      0.07       118

avg / total       0.98      0.49      0.64      6000

None
*********************************
Threshold: 0.1
Confusion Matrix:
      Prediction
         0     1
    0 5038   844
    1   27    91

Accuracy Score:
0.854833333333
()
Classification Report:
             precision    recall  f1-score   support

          0       0.99      0.86      0.92      5882
          1       0.10      0.77      0.17       118

avg / total       0.98      0.85      0.91      6000

None
*********************************
Threshold: 0.15
Confusion Matrix:
      Prediction
         0     1
    0 5651   231
    1   57    61

Accuracy Score:
0.952
()
Classification Report:
             precision

- According to the above thresholds and results, the threshold should be 0.25 due to the fact that it has the highest f1 score for 1, but in general 0.25 is really low compared to previous thresholds

In [80]:
score_cosine=[]
k=0
while(k<n):
    if(df_only_names.loc[k]['cosine_similarity_html']>0.25):
            score_cosine.append(1)
    else:
            score_cosine.append(0)
    k=k+1
print(analytics(df_only_names['label'], score_cosine))

Confusion Matrix:
      Prediction
         0     1
    0 5860    22
    1   93    25

Accuracy Score:
0.980833333333
()
Classification Report:
             precision    recall  f1-score   support

          0       0.98      1.00      0.99      5882
          1       0.53      0.21      0.30       118

avg / total       0.98      0.98      0.98      6000

None


### 3.1.2. Jaccard Similarity 

In [81]:
# inspired by http://billchambers.me/tutorials/2014/12/22/cosine-similarity-explained-in-python.html
import string
import math
from __future__ import division
def jaccard_similarity(query, document):
    intersection = set(query).intersection(set(document))
    union = set(query).union(set(document))
    return len(intersection)/len(union)

In [82]:
import codecs
import unicodedata
from bs4 import BeautifulSoup
i=0
keys=list(all_df['page_id'])
cols=['name','brand','type','color','itemCondition','availability','priceCurrency','price','megapixels','optical_zoom','video_resolution',
     'iso_range','unit','width','height','depth','unit2','value','model','has_wifi','has_bluetooth','battery']
tokenize = lambda doc: doc.lower().split(" ")
jaccard=[]
n=len(keys)
while(i<n):

    #get the corresponding html and parse it
    thing=(keys[i].encode('ascii','ignore'))+'.html'
    f=codecs.open(thing, 'r')
    text1=f.read()
    cleantext = BeautifulSoup(text1, "lxml").text
    as_str=cleantext.encode('ascii','ignore')
    clean=as_str.translate(None, '\r\n\t\,)};{$.|:(#"[]/><+=')
    html = denoise_text(clean)
    html=replace_contractions(html)

    #create one big string from the columns in the catalog
    ii=0
    a=all_df.loc[i]['name']
    while(ii<len(cols)):
        a=a+' '+all_df.loc[i][cols[ii]]
        ii=ii+1

    #tokenize them
    tokenized_html = tokenize(html) #page ids
    tokenized_gs = tokenize(a) #product catalog
    
    #eliminate tokens that are less than 46 length in html
    columns=list(filter(None, list(tokenized_html))) #eliminate emty values
    n1=len(columns)
    vocab2=[]
    for k in range(0, n1):
        if(len(columns[k])<46):
            vocab2.append(columns[k])
    

    jaccard.append(jaccard_similarity(vocab2,tokenized_gs))
    i=i+1

In [83]:
max(jaccard)

0.027923211169284468

In [84]:
df_only_names['jaccard_similarity_html']=jaccard

In [85]:
max(df_only_names['jaccard_similarity_html'])

0.027923211169284468

In [86]:
min(df_only_names['jaccard_similarity_html'])

0.0006485084306095979

In [87]:
#measure the accruacy for different levels
i=0
n=df_only_names.shape[0]
ls=list(np.arange(0.0007, 0.027, 0.001))
while(i<len(ls)):
    k=0
    score_cosine=[]
    while(k<n):
        if(df_only_names.loc[k]['jaccard_similarity_html']>ls[i]):
            score_cosine.append(1)
        else:
            score_cosine.append(0)
        k=k+1
    print('Threshold: {}'.format(ls[i]))
    print(analytics(df_only_names['label'], score_cosine))
    print('*********************************')
    i=i+1

Threshold: 0.0007
Confusion Matrix:
      Prediction
         0     1
    0    2  5880
    1    0   118

Accuracy Score:
0.02
()
Classification Report:
             precision    recall  f1-score   support

          0       1.00      0.00      0.00      5882
          1       0.02      1.00      0.04       118

avg / total       0.98      0.02      0.00      6000

None
*********************************
Threshold: 0.0017
Confusion Matrix:
      Prediction
         0     1
    0   41  5841
    1    0   118

Accuracy Score:
0.0265
()
Classification Report:
             precision    recall  f1-score   support

          0       1.00      0.01      0.01      5882
          1       0.02      1.00      0.04       118

avg / total       0.98      0.03      0.01      6000

None
*********************************
Threshold: 0.0027
Confusion Matrix:
      Prediction
         0     1
    0  267  5615
    1    0   118

Accuracy Score:
0.0641666666667
()
Classification Report:
             precision 

Threshold: 0.0207
Confusion Matrix:
      Prediction
         0     1
    0 5877     5
    1  115     3

Accuracy Score:
0.98
()
Classification Report:
             precision    recall  f1-score   support

          0       0.98      1.00      0.99      5882
          1       0.38      0.03      0.05       118

avg / total       0.97      0.98      0.97      6000

None
*********************************
Threshold: 0.0217
Confusion Matrix:
      Prediction
         0     1
    0 5878     4
    1  117     1

Accuracy Score:
0.979833333333
()
Classification Report:
             precision    recall  f1-score   support

          0       0.98      1.00      0.99      5882
          1       0.20      0.01      0.02       118

avg / total       0.97      0.98      0.97      6000

None
*********************************
Threshold: 0.0227
Confusion Matrix:
      Prediction
         0     1
    0 5878     4
    1  117     1

Accuracy Score:
0.979833333333
()
Classification Report:
             pre

- As can be seen above when jaccard calculated, the similarity yielded values between 0,0007 and 0,028. We defined the threshold with 0.0107. But we believe this low accuracy score is not acceptable, so it won't be accepted. 

In [88]:
score_cosine=[]
k=0
while(k<n):
    if(df_only_names.loc[k]['jaccard_similarity_html']>0.0107):
            score_cosine.append(1)
    else:
            score_cosine.append(0)
    k=k+1
print(analytics(df_only_names['label'], score_cosine))

Confusion Matrix:
      Prediction
         0     1
    0 5652   230
    1   80    38

Accuracy Score:
0.948333333333
()
Classification Report:
             precision    recall  f1-score   support

          0       0.99      0.96      0.97      5882
          1       0.14      0.32      0.20       118

avg / total       0.97      0.95      0.96      6000

None


## 3.2. TFIDF

### 3.2.1. Cosine Similarity

In [89]:
import codecs
import unicodedata
from bs4 import BeautifulSoup
i=0
keys=list(all_df['page_id'])
cols=['name','brand','type','color','itemCondition','availability','priceCurrency','price','megapixels','optical_zoom','video_resolution',
     'iso_range','unit','width','height','depth','unit2','value','model','has_wifi','has_bluetooth','battery']
cosine=[]
n=len(keys)
while(i<n):
    thing=(keys[i].encode('ascii','ignore'))+'.html'
    f=codecs.open(thing, 'r')
    text1=f.read()
    cleantext = BeautifulSoup(text1, "lxml").text
    as_str=cleantext.encode('ascii','ignore')
    clean=as_str.translate(None, '\r\n\t\,)};{$.|:(#"[]/><+=')
    sample = denoise_text(clean)
    sample=replace_contractions(sample)
    
    ii=0
    a=all_df.loc[i]['name']
    while(ii<len(cols)):
        a=a+' '+all_df.loc[i][cols[ii]]
        ii=ii+1
    
    df_try=pd.DataFrame([a,sample])
    
    
    vectorizer = CountVectorizer()
    matrix=vectorizer.fit_transform(df_try[0]).toarray()
    tfidf_r = tfidf_transformer.fit_transform(matrix)
    vocab=vectorizer.vocabulary_
    bags_words=pd.DataFrame([tfidf_r.toarray()[0],tfidf_r.toarray()[1]], columns=sorted(vocab))
    
    columns=list(filter(None,list(bags_words.columns.values))) 
    n1=len(columns)
    vocab2=[]
    for k in range(0, n1):
        if(len(columns[k])<46):
            vocab2.append(columns[k])
    bags_words2=bags_words[vocab2]
    cosine.append(pairwise.cosine_similarity(X=bags_words2.iloc[[0]] , Y=bags_words2.iloc[[1]])[0][0])
    i=i+1

In [90]:
df_only_names['cosine_similarity_html_tfidf']=cosine

In [91]:
max(cosine)

0.36333667039033735

In [92]:
min(cosine)

0.00027081267697391686

In [93]:
np.arange(0.0002, 0.363, 0.01)

array([  2.00000000e-04,   1.02000000e-02,   2.02000000e-02,
         3.02000000e-02,   4.02000000e-02,   5.02000000e-02,
         6.02000000e-02,   7.02000000e-02,   8.02000000e-02,
         9.02000000e-02,   1.00200000e-01,   1.10200000e-01,
         1.20200000e-01,   1.30200000e-01,   1.40200000e-01,
         1.50200000e-01,   1.60200000e-01,   1.70200000e-01,
         1.80200000e-01,   1.90200000e-01,   2.00200000e-01,
         2.10200000e-01,   2.20200000e-01,   2.30200000e-01,
         2.40200000e-01,   2.50200000e-01,   2.60200000e-01,
         2.70200000e-01,   2.80200000e-01,   2.90200000e-01,
         3.00200000e-01,   3.10200000e-01,   3.20200000e-01,
         3.30200000e-01,   3.40200000e-01,   3.50200000e-01,
         3.60200000e-01])

In [94]:
#measure the accruacy for different levels
i=0
n=df_only_names.shape[0]
ls=list(np.arange(0.0002, 0.363, 0.01))
while(i<len(ls)):
    k=0
    score_cosine=[]
    while(k<n):
        if(df_only_names.loc[k]['cosine_similarity_html_tfidf']>ls[i]):
            score_cosine.append(1)
        else:
            score_cosine.append(0)
        k=k+1
    print('Threshold: {}'.format(ls[i]))
    print(analytics(df_only_names['label'], score_cosine))
    print('*********************************')
    i=i+1

Threshold: 0.0002
Confusion Matrix:
      Prediction
         0     1
    0    0  5882
    1    0   118

Accuracy Score:
0.0196666666667
()
Classification Report:
             precision    recall  f1-score   support

          0       0.00      0.00      0.00      5882
          1       0.02      1.00      0.04       118

avg / total       0.00      0.02      0.00      6000

None
*********************************
Threshold: 0.0102
Confusion Matrix:
      Prediction
         0     1
    0  521  5361
    1    0   118

Accuracy Score:
0.1065
()
Classification Report:
             precision    recall  f1-score   support

          0       1.00      0.09      0.16      5882
          1       0.02      1.00      0.04       118

avg / total       0.98      0.11      0.16      6000

None
*********************************
Threshold: 0.0202
Confusion Matrix:
      Prediction
         0     1
    0 1808  4074
    1    0   118

Accuracy Score:
0.321
()
Classification Report:
             precision

None
*********************************
Threshold: 0.2002
Confusion Matrix:
      Prediction
         0     1
    0 5878     4
    1  102    16

Accuracy Score:
0.982333333333
()
Classification Report:
             precision    recall  f1-score   support

          0       0.98      1.00      0.99      5882
          1       0.80      0.14      0.23       118

avg / total       0.98      0.98      0.98      6000

None
*********************************
Threshold: 0.2102
Confusion Matrix:
      Prediction
         0     1
    0 5881     1
    1  104    14

Accuracy Score:
0.9825
()
Classification Report:
             precision    recall  f1-score   support

          0       0.98      1.00      0.99      5882
          1       0.93      0.12      0.21       118

avg / total       0.98      0.98      0.98      6000

None
*********************************
Threshold: 0.2202
Confusion Matrix:
      Prediction
         0     1
    0 5881     1
    1  106    12

Accuracy Score:
0.982166666667
(

- As can be seen above when cosine calculated, the similarity yielded values between 0,0002 and 0,1002. We defined the threshold with 0.363 and with this threshold we have f1 score of 1 as 0.34

In [95]:
score_cosine=[]
k=0
while(k<n):
    if(df_only_names.loc[k]['cosine_similarity_html_tfidf']>0.1002):
            score_cosine.append(1)
    else:
            score_cosine.append(0)
    k=k+1
print(analytics(df_only_names['label'], score_cosine))

Confusion Matrix:
      Prediction
         0     1
    0 5717   165
    1   61    57

Accuracy Score:
0.962333333333
()
Classification Report:
             precision    recall  f1-score   support

          0       0.99      0.97      0.98      5882
          1       0.26      0.48      0.34       118

avg / total       0.98      0.96      0.97      6000

None
